In [4]:
%pip install torch pandas torchvision scikit-learn tqdm kaggle yacs timm einops munch -q

Note: you may need to restart the kernel to use updated packages.


In [1]:
# upload kaggle.json first.
!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [1]:
# !apt update -qq
# !apt install -qq unzip
# !kaggle datasets download nirmalsankalana/sugarcane-leaf-disease-dataset
# !unzip -q sugarcane-leaf-disease-dataset.zip -d data
# !kaggle datasets download  pungliyavithika/sugarcane-leaf-disease-classification
# !unzip -q sugarcane-leaf-disease-classification.zip -d data2
!unzip -q mendeley.zip -d mendeley

In [3]:
import os
import shutil

import pandas as pd

# Define paths
data_root = "mendeley"
images_dir = os.path.join(data_root, "images")

# Create images directory if it doesn't exist
os.makedirs(images_dir, exist_ok=True)

# List to store image paths and labels
dataset = []

# Loop through each subfolder
for subfolder in os.listdir(data_root):
    subfolder_path = os.path.join(data_root, subfolder)

    # Ensure it's a directory
    if os.path.isdir(subfolder_path) and subfolder != "images":
        # Loop through images inside the subfolder
        for image in os.listdir(subfolder_path):
            old_image_path = os.path.join(subfolder_path, image)

            # Ensure it's a file (image)
            if os.path.isfile(old_image_path):
                # Define new image path in "data/images" directory
                new_image_path = os.path.join(images_dir, image)

                # If filename already exists, rename it to avoid conflicts
                if os.path.exists(new_image_path):
                    base, ext = os.path.splitext(image)
                    counter = 1
                    while os.path.exists(new_image_path):
                        new_image_path = os.path.join(images_dir, f"{base}_{counter}{ext}")
                        counter += 1

                # Move image
                shutil.move(old_image_path, new_image_path)

                # Append to dataset with updated path and original label
                dataset.append({"image_path": new_image_path, "label": subfolder})

        # Optionally remove empty subfolder after moving images
        os.rmdir(subfolder_path)

df = pd.DataFrame(dataset)
df = df.rename(columns={"image_path": "image_id"})
df["image_id"] = df["image_id"].str.replace("data/images/", "", regex=False)

from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
df["label"] = label_encoder.fit_transform(df["label"])

df.to_csv(os.path.join(data_root, "dataset.csv"), index=False)

label_mapping = dict(zip(label_encoder.classes_, label_encoder.transform(label_encoder.classes_)))

In [1]:
import pandas as pd
from dataset import build_dataset, build_loader

# path = f'{data_root}/images'
path = ''
data_root = "mendeley"
df = pd.read_csv(f'{data_root}/dataset.csv')

trd, testd, vald = build_dataset(df, path)

/home/ubuntu/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
train_loader, test_loader, val_loader = build_loader(trd, vald, 16)

In [3]:
from train import main
model = main(train_loader, val_loader, 'tswinf_mendeley.pth')

2025-04-27 20:16:54.857412: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-04-27 20:16:54.870851: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1745785014.886635    6163 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1745785014.891740    6163 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-04-27 20:16:54.907757: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

[2025-04-27 20:16:56 CSWin_BOAT_64_24322_small_224](train.py 71): INFO Creating model:cswin_boat/CSWin_BOAT_64_24322_small_224


/home/ubuntu/.local/lib/python3.10/site-packages/timm/models/layers/__init__.py:48: FutureWarning: Importing from timm.models.layers is deprecated, please import via timm.layers
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.layers", FutureWarning)


[2025-04-27 20:17:00 CSWin_BOAT_64_24322_small_224](train.py 80): INFO number of params: 60235181
[2025-04-27 20:17:00 CSWin_BOAT_64_24322_small_224](train.py 101): INFO Start training
[2025-04-27 20:22:17 CSWin_BOAT_64_24322_small_224](train.py 205): INFO Train: [0/35]	eta 0:00:02 lr 0.000041	time 3.0006 (2.5177)	loss 1.5829 (1.5963)	grad_norm 2.1137 (1.6061)	mem 6408MB
[2025-04-27 20:22:17 CSWin_BOAT_64_24322_small_224](train.py 221): INFO EPOCH 0 training takes 0:05:17
[2025-04-27 20:23:24 CSWin_BOAT_64_24322_small_224](train.py 310): INFO TEST:	Time 1.976 (2.172)	Loss 1.5114 (1.5939)	Acc@1 18.750 (27.419)	Acc@5 100.000 (100.000)	Mem 6408MB
[2025-04-27 20:23:24 CSWin_BOAT_64_24322_small_224](train.py 318): INFO  * Acc@1 27.419 Acc@5 100.000
[2025-04-27 20:23:24 CSWin_BOAT_64_24322_small_224](train.py 322): INFO 
Classification Report:
              precision    recall  f1-score   support

           0     0.5926    0.1600    0.2520       100
           1     0.2632    0.0556    0.09

/usr/lib/python3/dist-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


[2025-04-27 20:23:25 CSWin_BOAT_64_24322_small_224](train.py 124): INFO tswinf_mendeley.pth saved !!!
[2025-04-27 20:23:25 CSWin_BOAT_64_24322_small_224](train.py 128): INFO Max accuracy: 27.42%
[2025-04-27 20:28:49 CSWin_BOAT_64_24322_small_224](train.py 205): INFO Train: [1/35]	eta 0:00:02 lr 0.000081	time 2.9767 (2.5773)	loss 1.5284 (1.5729)	grad_norm 1.8274 (2.5149)	mem 6408MB
[2025-04-27 20:28:49 CSWin_BOAT_64_24322_small_224](train.py 221): INFO EPOCH 1 training takes 0:05:24
[2025-04-27 20:29:56 CSWin_BOAT_64_24322_small_224](train.py 310): INFO TEST:	Time 2.440 (2.157)	Loss 1.6001 (1.5732)	Acc@1 18.750 (25.605)	Acc@5 100.000 (100.000)	Mem 6408MB
[2025-04-27 20:29:56 CSWin_BOAT_64_24322_small_224](train.py 318): INFO  * Acc@1 25.605 Acc@5 100.000
[2025-04-27 20:29:56 CSWin_BOAT_64_24322_small_224](train.py 322): INFO 
Classification Report:
              precision    recall  f1-score   support

           0     0.0000    0.0000    0.0000       104
           1     0.0000    0.00

KeyboardInterrupt: 